In [ ]:
!pip install torch transformers sentencepiece numpy jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.1 MB/s eta 0:00:00


To securely use your Hugging Face token, please add it to Colab's secrets manager. Click on the '🔑' icon in the left sidebar, then click 'Add new secret'. Name the secret `HF_TOKEN` and paste your Hugging Face token as the value. After adding it, run the following cell to retrieve and set your token.

In [ ]:
# Import the necessary library to retrieve secrets
from google.colab import userdata

# Retrieve the Hugging Face token from Colab secrets
hf_token = userdata.get('HF_TOKEN')

# Set the token as an environment variable so Hugging Face libraries can find it
import os
os.environ['HF_TOKEN'] = hf_token

print("Hugging Face token successfully loaded and set.")


Hugging Face token successfully loaded and set.


Now that your token is set, let's try loading the model again.

In [ ]:
def run_coref(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    return tokens, predictions[0].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

MODEL_NAME = "dslim/bert-base-NER"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
model.eval()

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

**Reasoning**:
The previous code cell successfully loaded the 'dslim/bert-base-NER' model without any `RepositoryNotFoundError`, which confirms that the model loading mechanism is working. The output of the cell displayed the model architecture, indicating a successful load.



In [ ]:
print("The model `dslim/bert-base-NER` has been successfully loaded using AutoTokenizer.from_pretrained and AutoModelForTokenClassification.from_pretrained. This confirms that the loading mechanism is functional with a valid Hugging Face model ID.")

The model `dslim/bert-base-NER` has been successfully loaded using AutoTokenizer.from_pretrained and AutoModelForTokenClassification.from_pretrained. This confirms that the loading mechanism is functional with a valid Hugging Face model ID.


In [ ]:
def run_coref(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    return tokens, predictions[0].tolist()

In [ ]:
def extract_mentions(tokens, predictions):
    mentions = []
    current = []

    for token, label in zip(tokens, predictions):
        if label > 0:
            current.append(token.replace("##", ""))
        else:
            if current:
                mentions.append(" ".join(current))
                current = []

    if current:
        mentions.append(" ".join(current))

    return mentions

In [ ]:
def build_chains(mentions):
    chains = []
    used = set()

    for m in mentions:
        if m.lower() in used:
            continue
        chain = [m]
        used.add(m.lower())

        for other in mentions:
            if other.lower() != m.lower() and other.lower()[:3] == m.lower()[:3]:
                chain.append(other)
                used.add(other.lower())

        chains.append(chain)

    return chains

In [ ]:
text = "Sarah bought a new car. She drove it to work. The vehicle performed excellently."

tokens, preds = run_coref(text)
mentions = extract_mentions(tokens, preds)
chains = build_chains(mentions)

print("Mentions:", mentions)
print("Chains:", chains)

Mentions: ['Sarah']
Chains: [['Sarah']]


#Gold Standard Annotation (Manual)

In [ ]:
gold = [
    ["Sarah", "She"],
    ["a new car", "it", "The vehicle"]
]

#Evaluation Metric

In [ ]:
def muc(gold, pred):
    correct = 0
    total = 0

    for g in gold:
        total += len(g) - 1
        for i in range(1, len(g)):
            for p in pred:
                if g[0] in p and g[i] in p:
                    correct += 1

    return correct / total if total else 0

In [ ]:
def ceaf(gold, pred):
    score = 0
    for g in gold:
        best = max(len(set(g) & set(p)) for p in pred)
        score += best
    return score / sum(len(g) for g in gold)

In [ ]:
print("MUC:", muc(gold, chains))
print("CEAF:", ceaf(gold, chains))

MUC: 0.0
CEAF: 0.2


#Visualization (HTML – Modern & Clean)

In [ ]:
def visualize(text, chains):
    colors = ["#FFD700", "#90EE90", "#ADD8E6", "#FFB6C1"]
    html = text

    for i, chain in enumerate(chains):
        for mention in chain:
            html = html.replace(
                mention,
                f"<span style='background:{colors[i % len(colors)]}'>{mention}</span>"
            )

    return html

In [ ]:
from IPython.display import HTML
HTML(visualize(text, chains))